In [2]:
import numpy as np
import pydicom as pyd
import os
import matplotlib.pyplot as plt
from glob import glob
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import scipy.ndimage
from skimage import morphology
from skimage import measure
from skimage.transform import resize
from sklearn.cluster import KMeans
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.tools import FigureFactory as FF
from plotly.graph_objs import *
init_notebook_mode(connected=True) 


In [3]:
data_path='./Lung-DICOM'
output_path='../Project'


In [4]:
# this function is fixing the missing values (if any) in the Slice Thickness Metadata
def load_scan(path):#We can also do it for the other code, do it later.
    # replace it with os.walk
    slices=[pyd.dcmread(path+'/'+s) for s in os.listdir(path)]#List comprehension
    #Sort the list by Instance number, instance is a unique key in each of the DICOM file
    slices.sort(key=lambda x: int(x.InstanceNumber)) 
    try:#Slice thickness is along z axis, but we dont know actual coordinates because of reference issues, so 
        #we define slice_thickness as difference.
        silce_thickness=np.abs(slices[0].ImagePositionPatient[2]-slices[1].ImagePositionPatient[2])
    except:
        slice_thickness=np.abs(slices[0].SliceLocation - slices[1].SliceLocation)

        
        # Slice location and image position [2] are same, this error handler ensures that if there is a missing value in image position, it takes slice location    for s in slices:#For every slice it will set the attribute to the value of above array
        s.SliceThickness=slice_thickness
    return slices

    
def get_pixels_hu(scans):
    image=np.stack(list(ds.pixel_array for ds in scans))#We have stacked all the arrays(pixel arrays) as one. 
    image=image.astype(np.int16)#Image is the' combined effect' of slices' pixels data
    intercept=scans[0].RescaleIntercept# These two attributes are already present in ds(ResInter and Slope)
    slope=scans[0].RescaleSlope
    image[image == -2000] = 0 #
    if slope !=1:#slope is assumed to be one generally
        image = slope * image.astype(np.float64)
        image=image.astype(np.int16)
    image=image+np.int16(intercept) #Formula for hu conversion: slope*image+intercept
    return np.array(image, dtype=np.int16)
id=0    

In [5]:
patient = load_scan(data_path)
imgs = get_pixels_hu(patient)# for the second function patient is scans
# It should be noted that imgs contain pixel data and array used.


FileNotFoundError: [WinError 3] The system cannot find the path specified: './Lung-DICOM'

In [ ]:
# 0print(patient)

In [ ]:
# print(imgs)
#This has loaded all the arrays within the pixel array

In [ ]:
output_path='../Project'
np.save(output_path+'/'+'PROJECTDATA.id_%d'%(id),imgs)
print('PROJECTDATA.id_%d'%(id))

In [ ]:
#Let's see graphically the statistics of bones and fires :p :
files_saved=(np.load('PROJECTDATA.id_%d.npy'%(id)))
imgs_plot=files_saved.astype(np.float64)
plt.hist(imgs_plot.flatten(), bins=50, color='c')
plt.xlabel("Hounsfield Units (HU)")
plt.ylabel("Frequency")
plt.show()

In [ ]:
print(patient[0].PixelSpacing[0])


In [ ]:
# printing out some parameters:
print('Pixel spacing(row,col):(%f,%f)'%(patient[0].PixelSpacing[0],patient[0].PixelSpacing[1]))
print('Slice Thickness: %f'%(patient[0].SliceThickness))
# we have rows=512, columns=512 so each represent approx 496 mm of length.
# each slice represents approximately 496 mm of data in length and width.
# if we scanned an 85-pound patient at the same “zoom” as a 190-pound patient, you wouldn’t want the scan 
# to occupy only the middle 250 voxels with a wide rim of air – you’d want to zoom in at the time of
# acquisition so that it makes a full use of the 512 x 512. This means that each CT scan actually 
# represents different dimensions in real life even though they are all 512 x 512 x Z slices.
print(patient[0].SliceThickness)
print((patient[0].PixelSpacing))

In [ ]:
# An isometric transformation is one where the distance between any two pixels in A, and the distance between the 
# equivalent two pixels in B does not change after the transformation has been applied
# There is always zoom/slice thickness invariance. A slice might have [2.4,3,3] as pixel spacing coordinates for one
# scan and for other it might be different. So we need to resample data by setting Pixel Spacing as [1,1,1]
id = 0

def resample(imgs_rsmpl,scan,set_spacing=[1,1,1]):
    # Determine current pixel spacing
    
    spacing = list(map(float,list([scan[0].SliceThickness])+list(scan[0].PixelSpacing)))#We are adding two
#   lists here. The final concatenated list contain spacing as this order: z,x,y,(,slices,rows,col)
    print(spacing) 
    spacing = np.array(spacing)#To make an array 
    print(imgs_rsmpl.shape)
    resize_factor = spacing / set_spacing   #
    new_real_shape = imgs_rsmpl.shape * resize_factor    
    new_shape = np.round(new_real_shape)
    real_resize_factor = new_shape / imgs_rsmpl.shape
    set_spacing = spacing / real_resize_factor
    print(set_spacing)
    
    imgs_rsmpl = scipy.ndimage.interpolation.zoom(imgs_rsmpl, real_resize_factor)
    
    return imgs_rsmpl, set_spacing

print ("Shape beforeresampling\t", files_saved.shape)
imgs_after_resamp, spacing = resample(files_saved, patient, [1,1,1])
print ("Shape after resampling\t", imgs_after_resamp.shape)
# Various print commands are executed above to make sure what we are doing is right.:p

In [ ]:
def make_mesh